In [1]:
!python -m pip install ani_file pillow colour

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# jupyter junk
def showm(images):
    for i in images:
        display(i)

In [3]:
from ani_file import ani_file
from itertools import cycle
from random import shuffle
from PIL import ImageFont, ImageDraw, Image
import numpy as np
from colour import Color
from io import BytesIO
import struct, builtins

In [27]:
def draw_letter(image, letter):
    im = image.copy()
    draw = ImageDraw.Draw(im)
    font = ImageFont.load("font/tom-thumb.pil")
    draw.text((26, 26), letter, font=font, fill="#000")
    return im

def draw_msg(image, msg):
    seq = zip(cycle([image]), msg)
    return [draw_letter(image, ch) for image, ch in seq]

def space_text(text, n):
    return (" " * n).join(list(text)) + (" " * n)

def replace_color(image, color):
    pxs = np.array(image)
    r, g, b, a = pxs.T
    key = (r == 0) & (g == 0) & (b == 0) & (a == 255)
    pxs[key.T] = color
    return Image.fromarray(pxs)

def paint_gradient(images, color_a, color_b):
    conv_color  = lambda color: (*[int(x * 255) for x in color.get_rgb()], 255)
    gradient = Color(color_a).range_to(Color(color_b), len(images))
    colors   = map(conv_color, gradient)
    return [replace_color(image, color) for image, color in zip(images, colors)]

def ico(image):
    buf = BytesIO()
    image.save(buf, format="ico", bitmap_format="bmp", sizes=[(32, 32)])
    buf.seek(0)
    return buf

def msg_img_seq(msg, image, color_a="#FF00FF", color_b="#FF0000", spacing=4):
    seq = paint_gradient(
        draw_msg(
            image, 
            space_text(msg, spacing)
        ), color_a, color_b)
    icos = map(ico, seq)
    return list(icos)

class my_ani_write(ani_file.ani_write):
    def _pack_anih(self):
        self._datawritten += 44 #Size of 11I of anih chunk
        return struct.pack("<4s10I",b"anih",36,36,self._nFrames,self._nSteps,self._iWidth,self._iHeight,self._iBitCount,self._nPlanes,self._iDispRate,self._bfAttributes)

    def _pack_info(self):
        if hasattr(self,"_inam") or hasattr(self,"_iart"):
            inamChunk,iartChunk = b"",b""
            # IMPORTANT: _inam and _iart need to be padded to even length for Chunk to work
            if hasattr(self,"_inam"):
                inamChunk = struct.pack(f'<4sI{len(self._inam)}s{"x"*(len(self._inam)%2)}' ,b"INAM",len(self._inam),self._inam)
            if hasattr(self,"_iart"):
                iartChunk = struct.pack(f'<4sI{len(self._iart)}s{"x"*(len(self._iart)%2)}',b"IART",len(self._iart),self._iart)
            
            self._datawritten += 12 + len(iartChunk) + len(inamChunk)
            return struct.pack("<4sI4s", b"LIST",4+len(inamChunk)+len(iartChunk),b"INFO") + inamChunk + iartChunk

    def _pack_frames(self):
        print(self.__dict__)
        iconSize = 0
        iconChunks = b""
        for icon in self._framespath:
            data = icon.read()
            iconChunks += struct.pack("<4sI", b"icon", len(data)) + data
            iconSize += 8 + len(data)
        self._datawritten += 12 + iconSize
        return struct.pack("<4sI4s", b"LIST", 4+iconSize, b"fram") + iconChunks

In [28]:
FLAG = "ugra_what_a_dummy_flag_1238098fffeea"
cur = Image.open('cursor.png')
seq = msg_img_seq(FLAG, cur)
seq_idx = list(range(len(seq)))
# shuffle(seq_idx); shuffle(seq_idx)
rate = [2 for _ in seq_idx]

with open("test.ico", "wb") as f: f.write(seq[0].read())

test = my_ani_write("test.ani")
test.setframespath(seq)
test.setseq(tuple(seq_idx))
test.setrate(rate)
test.setauthor("voronbay@uniiu.su")
test.setname("Pautinka 001 (06.05.1997)")
test.close()

{'_i_opened_the_file': <_io.BufferedWriter name='test.ani'>, '_file': <_io.BufferedWriter name='test.ani'>, '_nFrames': 180, '_nSteps': 180, '_iDispRate': 8, '_bfAttributes': 1, '_iWidth': 0, '_iHeight': 0, '_iBitCount': 0, '_nPlanes': 0, '_framespath': [<_io.BytesIO object at 0x7f32903a1120>, <_io.BytesIO object at 0x7f32b5de3ce0>, <_io.BytesIO object at 0x7f32911fe700>, <_io.BytesIO object at 0x7f3290308a40>, <_io.BytesIO object at 0x7f329030bd30>, <_io.BytesIO object at 0x7f329030bb50>, <_io.BytesIO object at 0x7f329030bf10>, <_io.BytesIO object at 0x7f3290308810>, <_io.BytesIO object at 0x7f32903e12b0>, <_io.BytesIO object at 0x7f32903e1490>, <_io.BytesIO object at 0x7f32903e1850>, <_io.BytesIO object at 0x7f32903e11c0>, <_io.BytesIO object at 0x7f32903e24d0>, <_io.BytesIO object at 0x7f32903e0090>, <_io.BytesIO object at 0x7f32903e0f40>, <_io.BytesIO object at 0x7f32903e0130>, <_io.BytesIO object at 0x7f32903e2430>, <_io.BytesIO object at 0x7f32903e1df0>, <_io.BytesIO object at 0x

In [7]:
from ani_file import ani_file
f = ani_file.open('/Users/v/Downloads/gam1571.ani', 'rb')
f.getrate()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/v/Downloads/gam1571.ani'